Será que podemos comparar ideologias políticas a partir de tweets?

ideologia: conjunto de convicções filosóficas, sociais, políticas etc. de um indivíduo ou grupo de indivíduos.

Comparações:
    - análise gráfica comparativa da frequência de palavras ou relacionadas a tópicos: 
        religião, educação, orientação sexual, identidade de gênero, economia, saúde, porte de armas, combate a pobreza
            ex: Quantidade de tweets do Bolsonaro falando sobre drogas é muito maior que a do Lula, que mostra maior favor a política anti-drogas
            (Buscar se a quantidade de palavras segue a mesma tendência)

    - Comparação percentual entre palavras específicas em relação ao total

    -! análise comparativa de tweets antes e depois de eleições

Observações:
    - contam retweets? -> Usar a função clean() para filtrar

Proximos passos:
Otimizar função de limpeza -> lemanização e estemização para remoção de radicais
Comparar temas por datas específicas

In [ ]:
# Imports section
import json
import spacy
import re
import pandas as pd
import unicodedata as uni

In [ ]:
def get_data():
    '''Open and read dataset files, saving contents as raw data.'''
    with open('dataset/jairbolsonaro.json','r') as file_01:
        data_01=json.load(file_01)
    with open('dataset/LulaOficial.json','r') as file_02:
        data_02=json.load(file_02)
        
    return data_01,data_02
# get_data()[0][0]

In [ ]:
def extract_tweets():
    '''Store tweets in lists without metadata'''
    data_01,data_02 = get_data()
    bolso_tweets = []
    for el in data_01:
        bolso_tweets.append(el['full_text'])

    lula_tweets = []
    for el in data_02:
        lula_tweets.append(el['full_text'])
        
    return bolso_tweets, lula_tweets

In [ ]:
def compare(bolso_tweets,lula_tweets,subject):
    # Fix_01:Currently isn't useful because it is too restricted, doesn't account for synonyms and related words
     
    '''Compare tweets from both sides based on a subject and return a dataframe for comparison'''
    # Look into Bolsonaro's tweets for the subject and store tweets in a list
    b_selected = []
    for tweet in bolso_tweets:
        if subject.lower() in tweet.lower():
            b_selected.append(tweet)

    # Look into Lula's tweets for the subject and store tweets in a list
    l_selected = []
    for tweet in lula_tweets:
        if subject.lower() in tweet.lower():
            l_selected.append(tweet)

    # Create two series based on the selected tweets lists
    b = pd.Series(b_selected,name='Bolsonaro')
    l = pd.Series(l_selected,name='Lula')

    # Create dataframe concatenating the two series
    pd.set_option('display.max_colwidth', None)
    df=pd.concat([b,l],axis=1)
    
    return df

In [ ]:
def lemmanize(tweet):
    '''Lemmanization fuction. Remove radicals of words, grouping them together for analysis.'''
    nlp=spacy.load('pt_core_news_sm')
    result=nlp(tweet)
    
    # returns as a string for use in regular expression matching
    return ' '.join([token.lemma_ for token in result])


In [ ]:
def clean(tweet):
    '''Removes radicals of words to group them up, normalizes every word, removes links, hashtags, mentions, pontuation, emogis and removes stopwords from a single tweet and returns a list with the remaining words'''
    
    stopwords=('de', 'a', 'o', 'que', 'd','e', 'do', 'da', 'em', 'um', 'nao', 'para', 'e', 'com', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 'ha', 'nos', 'ja', 'esta', 'eu', 'também', 'so', 'pelo', 'pela', 'ate', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estao', 'voce', 'tinha', 'foram', 'essa', 'num', 'nem', 'suas', 'meu', 'minha', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'sera', 'tenho', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'voces', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'esta', 'estamos', 'estao', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estavamos', 'estavam', 'estivera', 'estiveramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivessemos', 'estivessem', 'estiver', 'estivermos', 'estiverem', 'hei', 'ha', 'havemos', 'hao', 'houve', 'houvemos', 'houveram', 'houvera', 'houveramos', 'haja', 'hajamos', 'hajam', 'houvesse', 'houvessemos', 'houvessem', 'houver', 'houvermos', 'houverem', 'houverei', 'houvera', 'houveremos', 'houverao', 'houveria', 'houveriamos', 'houveriam', 'sou', 'somos', 'sao', 'era', 'eramos', 'eram', 'fui', 'foi', 'fomos', 'foram', 'fora', 'foramos', 'seja', 'sejamos', 'sejam', 'fosse', 'fossemos', 'fossem', 'for', 'formos', 'forem', 'serei', 'sera', 'seremos', 'serao', 'seria', 'seriamos', 'seriam', 'tenho', 'tem', 'temos', 'tem', 'tinha', 'tinhamos', 'tinham', 'tive', 'teve', 'tivemos', 'tiveram', 'tivera', 'tiveramos', 'tenha', 'tenhamos', 'tenham', 'tivesse', 'tivessemos', 'tivessem', 'tiver', 'tivermos', 'tiverem', 'terei', 'tera', 'teremos', 'terao', 'teria', 'teriamos', 'teriam')
    
    #lemmanize tweets in order to remove the radicals of words.
    tweet=lemmanize(tweet)
    
    # normalize text and make all letters lowercase
    tweet = uni.normalize('NFD', tweet).encode('ASCII', 'ignore').decode('utf-8').lower()
    
    # remove links
    url_pattern=r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'\".,<>?«»“”‘’]))'
    
    # remove hashtags
    tags_pattern=r'\#([a-zA-Z0-9_]{1,50})'
    
    # remove mentions
    mentions_pattern=r'\@([a-zA-Z0-9_]{1,50})'    
    
    # remove ponctuation and emogis 
    rest_pattern=r'[^\w\s]'
    
    # apply all patterns deleting all matched objects
    patterns = [url_pattern,tags_pattern,mentions_pattern,rest_pattern]
    for pattern in patterns:
        tweet = re.sub(pattern,'',tweet)
    
    clean_tweet=[word for word in tweet.split() if word not in stopwords]
    
    return clean_tweet


In [ ]:
def count_words(tweets,min=0):
    '''Count the ocurrence of key words in the tweets'''
    words={}
    selected_words=[]
    for tweet in tweets:
        for word in clean(tweet):
            if word in words.keys():
                words[word] += 1
            else:
                words[word] = 1

                
    # Filter the words into a list based on a minimum number of ocurrences
    for word, ocurrence in words.items():
        if ocurrence > min:
            selected_words.append(word)

    # return selected_words
    return sorted(words,key=words.get,reverse=True)

In [ ]:
bolso_tweets, lula_tweets = extract_tweets()
# count_words(bolso_tweets)
print(clean(bolso_tweets[1]))